Começamos removendo o arquivo `Iris.csv`, se ele já existe.


In [ ]:
!rm -rf Iris.csv

Aqui criamos uma interface para upload do arquivo `Iris.csv` (assumimos que o arquivo vai ser enviado exatamente com esse nome). Você pode baixar esse arquivo [daqui deste link](https://drive.google.com/uc?id=1d3NbjXro_BfnYpFm66ETBfe7ubAZPAoL).

In [ ]:
from google.colab import files
import io

# A variável upload é um dicionário com todos
# arquivos que foram enviados
uploaded = files.upload()

# A variável f é o arquivo de nome Iris.csv
# buscado no dicionário acima.
f = io.BytesIO(uploaded['Iris.csv'])

Saving Iris.csv to Iris.csv


Nas linhas abaixo zeramos o cursor do arquivo e lemos todas as linhas do arquivo em `lines`.

In [ ]:
# Zeramos o cursor para garantir que a leitura
# do arquivo inicie do início
f.seek(0)

# A variável lines é uma lista que contém as strings
# que representam cada linha do arquivo lido.
lines = f.readlines()

Abaixo criamos as matrizes dos dados de entrada e saída correspondentes, vazias mas já com as dimensões corretas.

In [ ]:
import numpy as np

# Aqui criamos as matrizes de entradas e saídas correspondentes
# ainda vazias
X = np.zeros((len(lines)-1,4)) # 4 entradas
Y = np.zeros((len(lines)-1,3)) # 3 saídas (one-hot encoding)

# Teremos 3 categorias. O vetor abaixo lista as strings
# que representam as categorias possíveis
cat = np.array(['Iris-setosa','Iris-versicolor','Iris-virginica'])

No laço abaixo populamos as matrizes `X` e `Y`.

In [ ]:
# Para cada linha do arquivo, exceto
# a primeira linha que é o cabeçalho
for i, line in enumerate(lines[1:]):

  # Aqui decodificamos a linha para transformar
  # de binário para caracteres ascii, e descartamos
  # o último caractere que representa uma nova linha
  s = line.decode()[:-1]

  # Aqui separamos os dados por vírgulas,
  # descartando o primeiro valor que é o id
  # pois usaremos i do laço como id.
  _,sl,sw,pl,pw,sp = s.split(',')

  # Transformamos as strings que representam
  # as dimensões de sépala e pétala para ponto
  # flutuante.
  sl = float(sl)
  sw = float(sw)
  pl = float(pl)
  pw = float(pw)

  # Aqui populamos as matrizes X e Y com os dados
  # coletados.
  X[i:] = np.array([sl,sw,pl,pw])
  Y[i:] = (cat == sp).astype('float') # Atenção para essa linha!

  # A última linha acima merece uma explicação mais longa:
  # Nessa linha fazemos um teste booleano, comparando cada
  # elemento do vetor cat com a string daquela linha do arquivo.
  # Para os elementos onde a comparação der verdadeiro, teremos
  # um booleano True, e para os elementos onde a comparação
  # der falso, teremos um booleano False. O resultado da comparação
  # do vetor cat com a string sp, na expressão cat == sp resulta
  # numa array booleana com mesmo formato que cat, mas com
  # valores booleanos True ou False em cada posição. O método
  # .astype('float') transforma o resultado em um array de float,
  # com 1.0 representando True e 0.0 representando False.
  # Então transformamos uma string como 'Iris-setosa' no vetor
  # [1.0, 0.0, 0.0] e uma string como 'Iris-versicolor' no vetor
  # [0.0, 1.0, 0.0] e assim por diante.

Nas linhas abaixo embaralhamos as amostras para não causar nenhum tipo de tendência no treinamento.

In [ ]:
import random

# Aqui criamos uma lista de índices
# embaralhados
indexes = list(range(150))
random.shuffle(indexes)

# Essa variável T indica quantas amostras
# serão usadas para treinamento. As demais
# serão usadas para validação
T = 140

# Aqui preparamos as matrizes dos pares
# de dados de treinamento e validação.
Xt = np.zeros((T,4))
Yt = np.zeros((T,3))
Xv = np.zeros((150-T,4))
Yv = np.zeros((150-T,3))

# Aqui preenchemos as matrizes com os
# respectivos valores
for i in range(0,T):
  Xt[i,:] = X[indexes[i],:]
  Yt[i,:] = Y[indexes[i],:]
for i in range(0,150-T):
  Xv[i,:] = X[indexes[T+i],:]
  Yv[i,:] = Y[indexes[T+i],:]

O código abaixo implementa a rede neural com o método de backpropagation.

In [ ]:
# Essa será a função de ativação
# utilizada
def sigmoid(x):
  return 1.0 / (1.0 + np.exp(-x))

# Função softmax
def softmax(x):
  ex = np.exp(x)
  s = np.sum(ex)
  return ex/s

# O código abaixo implementa uma rede
# neural ariticial estilo perceptron
# com 4 entradas, 8 neurônios escondidos
# e 3 saídas
class Perceptron:
  def __init__(self):

    # Pesos e biases da entrada para a camada
    # escondida
    self.Wh = np.random.random((8,4))*2.0 - 1.0
    self.bh = np.random.random((8,1))*2.0 - 1.0

    # Pesos e bieases da camada escondida para
    # a saída
    self.Wo = np.random.random((3,8))*2.0 - 1.0
    self.bo = np.random.random((3,1))*2.0 - 1.0

    # Esse será o passo de aprendizagem
    self.eta = 0.1

  def forward(self,x):

    # Essa função faz o cálculo da saída
    # da rede neural no sentido direto.

    # Essa linha garante que x tenha tamanho
    # de 4 linhas e 1 coluna
    x = np.reshape(x,(4,1))

    # Calcula a soma ponderada para a camada escondida
    # somado ao bias
    self.sh = np.dot(self.Wh,x) + self.bh

    # Função de ativação é aplicada à camada escondida
    self.zh = sigmoid(self.sh)

    # Calcula a soma ponderada para a camada de saída
    # somado ao bias
    self.so = np.dot(self.Wo,self.zh) + self.bo

    # Função de ativação é aplciada à camada de saída
    self.zo = softmax(self.so)
    return self.zo

  def train(self, X, Y):
    # Essa função faz um ciclo de
    # treinamento em todos os dados
    # dos pares X e Y

    # A variável Err é o erro acumulado
    # que só serve para avaliar a qualidade
    # final da rede neural. Na prática não
    # precisamos dele para o treinamento
    Err = 0.0

    # Na linha abaixo obtemos a quantidade
    # total de pares
    total = X.shape[0]

    # Laço de treinamento conforme
    # o erro de cada par
    for i in range(total):

      # Capturamos o vetor de entradas do par
      x = X[i,:]
      x = np.reshape(x,(4,1))

      # Capturamos o vetor de saída do par
      y_hat = Y[i,:]
      y_hat = np.reshape(y_hat,(3,1))

      # Fazemos o cálculo da saída da rede
      # neural
      y = self.forward(x)

      # Calculamos o erro médio, para avaliar
      # a evolução da performance. Isso não
      # é usado para calcular o ajuste dos
      # pesos e biases.
      err = - np.sum(y_hat*np.log(y))
      Err = Err + err

      # Aqui calculamos os deltas de trás para frente
      # no sentido inverso (daí o nome backpropagation)

      # Primeiro calculamos o delta do erro da saída
      # Aqui multiplicamos o erro pela derivada
      # da função de ativação na saída. A função
      # sigmoide possui uma derivada interessante.
      # se z é a sigmoide, a derivada é z*(1-z)
      self.do = (y - y_hat)

      # O delta da camada escondida é calculado
      # usando os pesos para propagar o delta do erro
      # da saída para o delta do erro da camada escondida
      self.dh = np.dot(self.Wo.T, self.do) \
                 * self.zh * (1.0 - self.zh)

      # Agora é só dar um passo, de tamanho eta, na
      # direção oposta do gradiente. Lembrando que
      # para os pesos, a derivada pessoal é gerada
      # computando a multiplicação do delta do erro
      # pela saída da camada anterior, gerando uma
      # matriz resultante da multiplicação desses
      # dois vetores. Para o bias basta usar o próprio
      # delta do erro.
      self.Wo = self.Wo - self.eta * np.dot(self.do,self.zh.T)
      self.bo = self.bo - self.eta * self.do
      self.Wh = self.Wh - self.eta * np.dot(self.dh,x.T)
      self.bh = self.bh - self.eta * self.dh

    # Já fora do laço, dividimos o erro pelo total
    # de amostras para normalizar
    Err = Err / total

    # Retornamos o erro
    return Err

Abaixo executamos o código, treinando a rede neural com os dados de treinamento separados anteriormente.

In [ ]:
# Criamos p que é nossa rede neural
p = Perceptron()

# Treinaremos 10 mil + 1 épocas
for i in range(10001):

  # Aqui um passo de treinamento
  Err = p.train(Xt,Yt)

  # A cada mil passos mostramos o erro
  # total para verificar se está mesmo
  # diminuindo.
  if not (i % 1000) or i == 0:
    print('Err = ',Err)

Err =  0.9472482260778524
Err =  0.07466243808344437
Err =  0.030113238248304874
Err =  0.02834979507977415
Err =  0.0271367514013101
Err =  0.02627194575903286
Err =  0.02548090925841597
Err =  0.023190586927961537
Err =  0.01888004416533082
Err =  0.01396996479385829
Err =  0.007650199816369704


Aqui vamos avaliar a performance dessa rede neural treinada, vendo como ela se sai nos dados separados para validação (dados para os quais ela nunca foi treinada).

In [ ]:
# Essa linha abaixo serve para suprimir a notação
# científica padrão do numpy para facilitar nosso
# olho humano comparar os resultados
np.set_printoptions(formatter={'float':lambda x: '%+01.2f ' % x})

# Para cada amostra de treinamento
# nesse laço
for i in range(150-T):

  # Separamos a entrada
  xv = Xv[i,:]

  # Calculamos a saída da rede
  y = p.forward(xv)

  # Separamos a saída esperada
  yv = Yv[i,:]

  # Mostramos ambos lado a lado
  print(y.T[0], yv)


[+0.00  +0.00  +1.00 ] [+0.00  +0.00  +1.00 ]
[+0.00  +1.00  +0.00 ] [+0.00  +1.00  +0.00 ]
[+0.00  +0.00  +1.00 ] [+0.00  +0.00  +1.00 ]
[+0.00  +0.00  +1.00 ] [+0.00  +0.00  +1.00 ]
[+0.00  +1.00  +0.00 ] [+0.00  +1.00  +0.00 ]
[+0.00  +1.00  +0.00 ] [+0.00  +1.00  +0.00 ]
[+1.00  +0.00  +0.00 ] [+1.00  +0.00  +0.00 ]
[+0.00  +0.00  +1.00 ] [+0.00  +0.00  +1.00 ]
[+0.00  +1.00  +0.00 ] [+0.00  +1.00  +0.00 ]
[+1.00  +0.00  +0.00 ] [+1.00  +0.00  +0.00 ]
